In [ ]:
# -------------------------------
# Passo 1: Instalar pacotes necessários
# -------------------------------
!pip install ultralytics -q
!pip install torchmetrics -q
!pip install torchvision -q
!pip install PyYAML -q

import torch, os, zipfile, yaml
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 17.5 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# -----------------------------------------
# Passo 2: Definir dispositivo (CPU ou GPU)
# -----------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando o dispositivo: {device}")

Usando o dispositivo: cpu


In [ ]:
# -------------------------------
# Passo 2: Montar Google Drive
# -------------------------------
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# -------------------------------
# Passo 3: Extrair o dataset robocup-v1-18.zip
# -------------------------------
zip_path = "/content/drive/MyDrive/robocup-v1-18.zip"
extract_path = "/content/robocup-v1-18"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"✅ Dataset extraído para: {extract_path}")

✅ Dataset extraído para: /content/robocup-v1-18


In [ ]:
# -------------------------------
# Passo 4: Listar classes a partir do data.yaml
# -------------------------------
yaml_path = os.path.join(extract_path, "data.yaml")

with open(yaml_path, "r") as f:
    data_yaml = yaml.safe_load(f)

classes = data_yaml["names"]

print(f"\n📊 Total de classes: {len(classes)}")
print("Lista de classes:")
for i, cls in enumerate(classes):
    print(f"{i}: {cls}")


📊 Total de classes: 18
Lista de classes:
0: AllenKey
1: Axis2
2: Bearing2
3: Drill
4: F20_20_B
5: F20_20_G
6: Housing
7: M20
8: M20_100
9: M30
10: Motor2
11: S40_40_B
12: S40_40_G
13: Screwdriver
14: Spacer
15: Wrench
16: container_box_blue
17: container_box_red


In [ ]:
# -------------------------------
# Passo 5: Criar modelo YOLO
# -------------------------------
# Começa com YOLOv8 nano (mais leve e rápido). Pode trocar por yolov8s.pt, yolov8m.pt, etc.
model = YOLO("yolov8n.pt")

In [ ]:
# -------------------------------
# Passo 6: Treinar o modelo
# -------------------------------
# results = model.train(
#     data=yaml_path,
#     epochs=3,
#     imgsz=320,
#     batch=8,
#     device=device
# )

DRIVE_PROJECT_PATH = "/content/drive/MyDrive/yolo-project"

results = model.train(
    data=yaml_path,   # usa o data.yaml do robocup-v1-18
    epochs=30,        # ajuste conforme necessário
    imgsz=640,        # tamanho das imagens
    device=device,    # usa GPU se disponível
    save_period=1,    # salva checkpoint na pasta "weights" a cada época
    patience=50,      # evita early stopping antes de 30 épocas
    project=DRIVE_PROJECT_PATH,  # Define o diretório *raiz* do projeto no Drive
    name='train_robocup',        # Nome da subpasta dentro de 'project' (e.g., .../yolo_robocup_training/train_robocup)
)

engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/robocup-v1-18/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train_robocup2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=50, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=/content/drive/MyDrive/yol

In [ ]:
# ---------------------------------------------------------------------
# Passo 6.5: Recuperar o Modelo do Checkpoint Salvo no Google Drive
# (Use este passo para substituir o treinamento interrompido)
# ---------------------------------------------------------------------
import os
from ultralytics import YOLO # Certifica-se de que YOLO está importado

# 1. Defina os caminhos usados para salvar o treinamento
# Confirme se este caminho corresponde ao que você usou na célula 6:
DRIVE_PROJECT_PATH = "/content/drive/MyDrive/yolo-project"
TRAINING_RUN_NAME = 'train_robocup2'

# 2. Defina o caminho COMPLETO para o arquivo de checkpoint
# Use 'last.pt' para o estado mais recente ou 'best.pt' para o modelo com melhor mAP.
CHECKPOINT_FILENAME = 'best.pt' # Altere para 'best.pt' se preferir o melhor resultado

checkpoint_path = os.path.join(
    DRIVE_PROJECT_PATH,
    TRAINING_RUN_NAME,
    'weights',
    CHECKPOINT_FILENAME
)

# 3. Carregue o modelo a partir do arquivo de checkpoint
model = YOLO(checkpoint_path)

print(f"✅ Modelo recuperado com sucesso de: {checkpoint_path}")
print("O objeto 'model' está pronto para ser usado nas células de validação e teste.")

✅ Modelo recuperado com sucesso de: /content/drive/MyDrive/yolo-project/train_robocup2/weights/best.pt
O objeto 'model' está pronto para ser usado nas células de validação e teste.


In [ ]:
# -------------------------------
# Passo 7: Validar o modelo e exibir métricas detalhadas (Formato Faster R-CNN)
# -------------------------------
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Validação
# O YOLO executa a validação no melhor modelo ('best.pt') por padrão
metrics = model.val()

# 2. Extração e Cálculo de Métricas Globais (para replicar o formato principal)
results_dict = metrics.results_dict
p = results_dict['metrics/precision(B)']
r = results_dict['metrics/recall(B)']
map50 = results_dict['metrics/mAP50(B)']
map50_95 = results_dict['metrics/mAP50-95(B)'] # mAP 50-95 (equivalente ao seu mAP 50-90)

# Cálculo do F1-score global
f1_score = 2 * (p * r) / (p + r) if (p + r) > 0 else 0

# Cálculo do Tempo por Época e FPS
# O tempo por época (time_per_epoch) está no log de treino (stdout) e deve ser analisado lá.
# Aqui focamos na velocidade de inferência (Speed)
inference_ms = metrics.speed['inference']
fps = 1000 / inference_ms

# 3. Exibição no formato Faster R-CNN (Principal)
print("\n--- Resultados de Performance (Formato Faster R-CNN) ---")
print(f"  map: {map50_95:.4f} (mAP 50-95)")
print(f"  map_50: {map50:.4f}")
print(f"  Precision (all): {p:.4f}")
print(f"  Recall (all): {r:.4f}")
print(f"  F1-score (all): {f1_score:.4f}")
print(f"  Velocidade de Inferência (FPS): {fps:.2f} (Baseado em {inference_ms:.2f} ms/imagem)")

# 4. Análise de Desempenho por Classe
class_names = list(metrics.names.values())
class_metrics = {}

for i, cls in metrics.names.items():
    class_metrics[cls] = {
        'Precision': metrics.box.p[i].item(),
        'Recall': metrics.box.r[i].item(),
        'mAP50': metrics.box.ap50[i].item(),
        'mAP50-95': metrics.box.ap[i].item()
    }

print("\n--- Análise de Desempenho por Classe ---")
df_class = pd.DataFrame.from_dict(class_metrics, orient='index')
# Incluindo a métrica geral no DataFrame
df_class.loc['all'] = {'Precision': p, 'Recall': r, 'mAP50': map50, 'mAP50-95': map50_95}
df_class = df_class[['Precision', 'Recall', 'mAP50', 'mAP50-95']]

# A saída 'to_string' replica uma tabela simples.
print(df_class.to_string(float_format="%.4f"))


# 5. Matriz de Confusão
print("\n--- Matriz de Confusão ---")

# O objeto ConfusionMatrix armazena os dados.
conf_matrix_obj = metrics.confusion_matrix

# Plotando a matriz de confusão (normalizada para melhor visualização)
# O Ultralytics tem uma função integrada de plotagem
conf_matrix_obj.plot(normalize=True, names=class_names, save_dir=metrics.save_dir)

print(f"\n✅ Matriz de Confusão plotada e salva em: {os.path.join(metrics.save_dir, 'confusion_matrix.png')}")
print("\n(Verifique o diretório de saída do treinamento para o arquivo PNG)")

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
Model summary (fused): 72 layers, 3,009,158 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 726.8±301.8 MB/s, size: 22.1 KB)
val: Scanning /content/robocup-v1-18/valid/labels... 1708 images, 119 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1729/1729 1.9Kit/s 0.9s
val: New cache created: /content/robocup-v1-18/valid/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 109/109 5.3s/it 9:38
                   all       1729       6933      0.986       0.99      0.992      0.923
              AllenKey        206        206      0.971      0.981      0.993      0.941
                 Axis2        433        439      0.986      0.986       0.99      0.875
              Bearing2        219        219      0.995      0.995      0.995      0.888
                 Drill        195        195      0.993      0.9

In [ ]:
# -------------------------------
# Passo 8: Testar várias imagens aleatórias e visualizar no Colab
# -------------------------------
import matplotlib.pyplot as plt
from PIL import Image
import os
import random

# Caminho das imagens de validação
test_images_path = os.path.join(extract_path, "valid/images")
all_images = os.listdir(test_images_path)

# Quantas imagens aleatórias mostrar
num_random_images = 5
random_images = random.sample(all_images, num_random_images)

for img_name in random_images:
    test_img = os.path.join(test_images_path, img_name)

    # Realiza a predição (salva resultados)
    results = model.predict(test_img, save=True)

    # Pega o diretório onde a imagem anotada foi salva
    save_dir = results[0].save_dir

    # A imagem salva tem o mesmo nome da original dentro da pasta save_dir
    saved_image_path = os.path.join(save_dir, os.path.basename(test_img))

    # Abre e mostra a imagem no Colab
    img = Image.open(saved_image_path)
    plt.figure(figsize=(8,8))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Predição YOLOv8 - {img_name}")
    plt.show()